<a href="https://colab.research.google.com/github/FraCorti/AI_Project/blob/master/spamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aggiungo l'installazione di **Kaggle** e lo scaricamento del dataset *spam.csv* 

In [8]:
!pip install kaggle
!mkdir /root/.kaggle
!echo '{"username":"fcorti","key":"fafa72652d9ae6ee3bab07bad5a1a96c"}' > /root/.kaggle/kaggle.json
!kaggle datasets download -d uciml/sms-spam-collection-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/208k [00:00<?, ?B/s]
100% 208k/208k [00:00<00:00, 33.6MB/s]


In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import layers
data = pd.read_csv('spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]


FileNotFoundError: ignored

In [0]:
ls

sample_data/  spam.csv


In [0]:
model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

In [0]:
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [0]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(32,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.train.AdamOptimizer(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

Instructions for updating:
Use tf.cast instead.


In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1000/1000 [==============================] - 0s 433us/sample - loss: 11.5490 - categorical_accuracy: 0.1080
Epoch 2/10
1000/1000 [==============================] - 0s 52us/sample - loss: 11.5025 - categorical_accuracy: 0.1130
Epoch 3/10
1000/1000 [==============================] - 0s 47us/sample - loss: 11.4853 - categorical_accuracy: 0.1070
Epoch 4/10
1000/1000 [==============================] - 0s 45us/sample - loss: 11.4737 - categorical_accuracy: 0.1090
Epoch 5/10
1000/1000 [==============================] - 0s 48us/sample - loss: 11.4735 - categorical_accuracy: 0.0980
Epoch 6/10
1000/1000 [==============================] - 0s 47us/sample - loss: 11.4722 - categorical_accuracy: 0.0800
Epoch 7/10
1000/1000 [==============================] - 0s 47us/sample - loss: 11.4714 - categorical_accuracy: 0.1250
Epoch 8/10
1000/1000 [==============================] - 0s 45us/sample - loss: 11.4703 - categorical_accuracy: 0.1120
Epoch 9